<a href="https://colab.research.google.com/github/citrus1998/nlp100/blob/main/nlp100_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://nlp100.github.io/data/neko.txt

--2022-06-16 22:30:21--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975789 (953K) [text/plain]
Saving to: ‘neko.txt’

neko.txt            100%[===================>] 952.92K  --.-KB/s    in 0.01s   

2022-06-16 22:30:21 (86.0 MB/s) - ‘neko.txt’ saved [975789/975789]



In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ub

In [ ]:
!mecab -o ./neko.txt.mecab ./neko.txt

In [ ]:
# Check the number of sentences
!wc -l ./neko.txt.mecab

226266 ./neko.txt.mecab


In [ ]:
!pip install mecab-python3 unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 28.0 MB/s 
     |████████████████████████████████| 47.4 MB 123 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=081c54fcb284ac5635da858077a2693603ae7b9abef9d60d1c30431e675ff2da
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


# 30. 形態素解析結果の読み込み  
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [ ]:
import MeCab

In [ ]:
filename = './neko.txt.mecab'

sentences = []
morphs = []
with open(filename, 'r', encoding='utf-8') as f:
  for l in f:
    if l not in 'EOS\n':
      spl_l = l.split('\t')
      if spl_l[0] != '':
        spl_cxt = spl_l[1].split(',')
        spl_cxt[-1] = spl_cxt[-1].replace('\n', '')
        morph = {'surface': spl_l[0], 'base': spl_cxt[6], 'pos': spl_cxt[0], 'pos1': spl_cxt[1]}
        morphs.append(morph)
    else:
      sentences.append(morphs)
      morphs = []

sentences = sentences[:2] + [m for m in sentences[2:] if len(m) != 0]
for i, m in enumerate(sentences[2]):
  print(m)

{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}
{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
{'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


#31.
動詞の表層形をすべて抽出せよ．

#32.
動詞の基本形をすべて抽出せよ．

#33.
2つの名詞が「の」で連結されている名詞句を抽出せよ．

In [ ]:
result31 = []
result32 = []
result33 = []

for s in sentences:
  for m in s:
    if m['pos'] == '動詞':
      result31.append(m['surface'])
      result32.append(m['base'])

  for i in range(1, len(s) - 1):
    if s[i]['surface'] == 'の' and (s[i-1]['pos'] == '名詞' and s[i+1]['pos'] == '名詞'):
      #print(s[i-1]['surface'], s[i+1]['surface'])
      result33.append((s[i-1]['surface'], s[i+1]['surface']))

result31 = set(result31)
result32 = set(result32)
result33 = set(result33)

4924


#34. 名詞の連接
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．